In [10]:
import os
import numpy as np
import tensorflow as tf

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

from tensorflow.keras.activations import relu
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Dropout, concatenate
from keras.utils.vis_utils import model_to_dot
#from tensorflow.keras.utils import model_to_do

from sklearn.metrics import classification_report
from IPython.display import SVG

## Utils Functions

In [11]:
def compute_metrics(raw_predictions, label_encoder):
    #convert raw predicition to class indexes
    threshold = 0.5
    class_predictions = [(x > threshold).astype(int) for x in model.predict(x_test)]
    
    #convert raw predicitions to class indexes
    threshold = 0.5
    class_predicitions = [(x > threshold).astype(int) for x in model.predict(x_test)]
    
    # select only one class
    # i.e., the dim in the vector with 1.0 all other are at 0.0
    class_index = ([np.argmax(x) for x in class_predictions])
    
    #convert back to original class names
    pred_classes = label_encoder.inverse_transform(class_index)
    
    #print precision, recall, f1-score report
    print(classification_report(y_test, pred_classes))

In [12]:
def load_fasttext_embeddings():
    glove_dir = './Datasets/glove.6B'
    embeddings_index = {}
    f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding='utf8')
    
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print("Found %s word vectors." % len(embeddings_index))
    return embeddings_index

In [13]:
def create_embeddings_matrix(embedding_index, vocabulary, embedding_dim = 100):
    embeddings_matrix = np.random.rand(len(vocabulary)+1, embedding_dim)
    
    for i, word in enumerate(vocabulary):
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector
    print("Matrix shape : {}" .format(embeddings_matrix.shape))
    return embeddings_matrix

In [14]:
def get_embeddings_layer(embeddings_matrix, name, max_len, trainable=False):
    embedding_layer = Embedding(
        input_dim=embeddings_matrix.shape[0],
        output_dim=embeddings_matrix.shape[1],
        input_length=max_len,
        weights=[embeddings_matrix],
        trainable=trainable,
        name=name)
    return embedding_layer

In [15]:
def get_conv_pool(x_input, max_len, sufix, n_grams=[3,4,5], feature_maps=100):
    branches = []
    for n in n_grams:
        branch = Conv1D(filters=feature_maps, kernel_size=n, activation=relu, name='Conv_'+sufix+'_'+str(n))(x_input)
        branch = MaxPooling1D(pool_size=max_len-n+1, strides=None, padding='valid', name='MaxPooling_'+sufix+'_'+str(n))(branch)
        branch = Flatten(name='Flatten_'+sufix+'_'+str(n))(branch)
        branches.append(branch)
    return branches

## Convolution Neural Net

In [16]:
def get_cnn_rand(embedding_dim=100, vocab_size=1000, max_len=50):
    # create the embedding layer
    embedding_matrix = np.random.rand(vocab_size, embedding_dim)
    embedding_layer = get_embeddings_layer(embedding_matrix, 'embedding_layer_dynamic', max_len, trainable=True)

    # connect the input with the embedding layer
    i = Input(shape=(max_len,), dtype='int32', name='main_input')
    x = embedding_layer(i)

    # generate several branches in the network, each for a different convolution+pooling operation,
    # and concatenate the result of each branch into a single vector
    branches = get_conv_pool(x, max_len, 'dynamic')
    z = concatenate(branches, axis=-1)
    z = Dropout(0.5)(z)

    # pass the concatenated vector to the predition layer
    o = Dense(1, activation='sigmoid', name='output')(z)

    model = Model(inputs=i, outputs=o)
    model.compile(loss={'output': 'binary_crossentropy'}, optimizer='adam')

    return model

In [17]:
model_1 = get_cnn_rand()

In [18]:
SVG(model_to_dot(model_1, show_layer_names=True, show_shapes=True).create(prog='dot', format='svg'))

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

## CNN-static

In [19]:
embeddings_index = load_fasttext_embeddings()

Found 400000 word vectors.


In [20]:
vocabulary = embeddings_index.keys()
# replace this by the vocabulary of the dataset you want to train
embeddings_matrix = create_embeddings_matrix(embeddings_index, vocabulary, 100)

Matrix shape : (400001, 100)


In [21]:
embedding_layer = get_embeddings_layer(embeddings_matrix, 'embedding_layer_static', 55, trainable=False)

In [22]:
def get_cnn_pre_trained_embeddings(embedding_layer, max_len):
    # connect the input with the embedding layer
    i = Input(shape=(max_len,), dtype='int32', name='main_input')
    x = embedding_layer(i)

    # generate several branches in the network, each for a different convolution+pooling operation,
    # and concatenate the result of each branch into a single vector
    branches = get_conv_pool(x, max_len, 'static')
    z = concatenate(branches, axis=-1)

    # pass the concatenated vector to the predition layer
    o = Dense(1, activation='sigmoid', name='output')(z)

    model = Model(inputs=i, outputs=o)
    model.compile(loss={'output': 'binary_crossentropy'}, optimizer='adam')

    return model

In [23]:
model = get_cnn_pre_trained_embeddings(embedding_layer, 55)

## Words vector are fined tuned

In [24]:
embedding_layer = get_embeddings_layer(embeddings_matrix, 'embedding_layer_dynamic', 55, trainable=True)
model = get_cnn_pre_trained_embeddings(embedding_layer, 55)

## CNN Multichannel

In [25]:
embedding_layer_channel_1 = get_embeddings_layer(embeddings_matrix, 'embedding_layer_dynamic', 55, trainable=True)
embedding_layer_channel_2 = get_embeddings_layer(embeddings_matrix, 'embedding_layer_static', 55, trainable=False)

def get_cnn_multichannel(embedding_layer_channel_1, embedding_layer_channel_2, max_len):

    # dynamic channel
    input_dynamic = Input(shape=(max_len,), dtype='int32', name='input_dynamic')
    x = embedding_layer_channel_1(input_dynamic)
    branches_dynamic = get_conv_pool(x, max_len, 'static')
    z_dynamic = concatenate(branches_dynamic, axis=-1)

    # static channel
    input_static = Input(shape=(max_len,), dtype='int32', name='input_static')
    x = embedding_layer_channel_2(input_static)
    branches_static = get_conv_pool(x, max_len, 'dynamic')
    z_static = concatenate(branches_static, axis=-1)

    # concatenate both models and pass to classification layer
    z = concatenate([z_static,z_dynamic], axis=-1)

    # pass the concatenated vector to the predition layer
    o = Dense(6, activation='sigmoid', name='output')(z)

    model = Model(inputs=[input_static, input_dynamic], outputs=o)
    model.compile(loss={'output': 'binary_crossentropy'}, optimizer='adam')

    return model

In [26]:
model = get_cnn_multichannel(embedding_layer_channel_1, embedding_layer_channel_2, 55)